In [1]:
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import math

In [2]:
restaurants = pd.read_json("SchemaOrg/all_cities.json", lines=True)

In [3]:
def unravel(x):
    return x[0] if type(x)==list else x

def unpackAddress(x, part="Street"):
    return x[part] if bool(x) else "null"

def reviewBody(x):
    if type(x)==list:
        body = ""
        for review in x:
            body += review['Body']
        return body.replace("\"", "")
    else: return x

def averageRating(x):
    if type(x)==list:
        ratingSum = 0
        count = 0
        for review in x:
            try:
                num = review['Rating']
                num = num.replace("\n", "")
                num = num.replace("\"", "")[0:1]
                num = int(num)
                ratingSum += num
                count += 1
            except:
                ratingSum = ratingSum
                count = count
        return ratingSum/len(x)
    else: return x   

In [4]:
#restaurants['Reviews'] = restaurants['Reviews'].apply(unravel)
restaurants['Address'] = restaurants['Address'].apply(unravel)

# unpack address
restaurants['Street'] = restaurants['Address'].apply(lambda x: unpackAddress(x, part='Street'))
restaurants['Locality'] = restaurants['Address'].apply(lambda x: unpackAddress(x, part='Locality'))
restaurants['Postalcode'] = restaurants['Address'].apply(lambda x: unpackAddress(x, part='Postalcode'))
restaurants['Region'] = restaurants['Address'].apply(lambda x: unpackAddress(x, part='Region'))
restaurants['Country'] = restaurants['Address'].apply(lambda x: unpackAddress(x, part='Country'))

# aggregate reviews
restaurants['ReviewCount'] = restaurants['Reviews'].apply(lambda x: len(x) if type(x)==list else x)
restaurants['ReviewBodies'] = restaurants['Reviews'].apply(reviewBody)
restaurants['AverageRating'] = restaurants['Reviews'].apply(averageRating)

# drop obsolete identifiers
restaurants.drop(labels=['Address'], axis=1, inplace=True)
restaurants.drop(labels=['RestaurantID'], axis=1, inplace=True)
restaurants.drop(labels=['Reviews'], axis=1, inplace=True)

# unify missing values; drop highly MV
restaurants = restaurants.replace("null", np.nan)
restaurants.drop(labels=['AcceptsReservations'], axis=1, inplace=True)

In [5]:
# TOO MANY MISSING VALUES

missing_proportion = np.round((restaurants.isnull().sum() / len(restaurants)),4)
display_MV = pd.DataFrame({'Attribute': restaurants.columns,
                                 'MV': missing_proportion})
display_MV

,Attribute,MV
City,City,0.0000
Cuisine,Cuisine,0.3008
Description,Description,0.6602
ImageURL,ImageURL,0.7639
Name,Name,0.0374
PaymentMethods,PaymentMethods,0.7729
Pricerange,Pricerange,0.6472
Telephone,Telephone,0.6328
Street,Street,0.5315
Locality,Locality,0.5478


In [6]:
# EXPERIMENTS YIELD THIS AS A CLEAN SAMPLE W.R.T. MISSING VALUES

sample = restaurants[(restaurants['Pricerange'] == '"$31 to $50"') | 
                     (restaurants['Pricerange'] == '"$30 and under"') | 
                     (restaurants['Pricerange'] == '"$50 and over"')
                    ] 

missing_proportion = np.round((sample.isnull().sum() / len(sample)),4)
display_MV = pd.DataFrame({'Attribute': sample.columns,
                                 'MV': missing_proportion})
display_MV

,Attribute,MV
City,City,0.0000
Cuisine,Cuisine,0.0000
Description,Description,0.0021
ImageURL,ImageURL,0.9996
Name,Name,0.0000
PaymentMethods,PaymentMethods,0.0021
Pricerange,Pricerange,0.0000
Telephone,Telephone,0.9996
Street,Street,0.0023
Locality,Locality,0.0023


In [7]:
# DROP UNUSABLE ATTRIBUTES

sample.drop(labels=['ImageURL', 'Telephone'], axis=1, inplace=True)

sample.head()

/Users/leonschuller/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,City,Cuisine,Description,Name,PaymentMethods,Pricerange,Street,Locality,Postalcode,Region,Country,ReviewCount,ReviewBodies,AverageRating
1131,charlotte,"""American""","""JP Charlotte is at the heart of The Westin Ch...","""JP Charlotte""","""AMEX, Carte Blanche, Diners Club, Discover, J...","""$30 and under""","""601 S College StreetCharlotte, NC 28202""","""Charlotte""","""28202""","""NC""","""United States""",40.0,We traveled to Uptown Charlotte for Easter Bru...,3.850
1132,charlotte,"""Steakhouse""","""Morton's The Steakhouse, the nation's premier...","""Morton's The Steakhouse - Charlotte""","""AMEX, Diners Club, MasterCard, Visa""","""$50 and over""","""227 W. Trade StreetCharlotte, NC 28202""","""Charlotte""","""28202""","""NC""","""United States""",40.0,"Our go-to for fine dining,steak and lobster. W...",4.450
1142,charlotte,"""American""","""Sullivan\u2019s is a vibrant neighborhood Ame...","""Sullivan's Steakhouse - Charlotte""","""AMEX, Diners Club, Discover, MasterCard, Visa""","""$31 to $50""","""1928 South BoulevardSuite 200Charlotte, NC 28...","""Charlotte""","""28203""","""NC""","""United States""",40.0,Waited a while to get main course and my steak...,3.925
1145,charlotte,"""American""","""Red Rocks Caf\u00E9 is a locally owned and op...","""Red Rocks Cafe - Charlotte""","""AMEX, MasterCard, Visa""","""$30 and under""","""4223-8 Providence RoadCharlotte, NC 28211""","""Charlotte""","""28211""","""NC""","""United States""",40.0,We dined outdoors and the device and good were...,4.375
1146,charlotte,"""Contemporary American""","""Bonterra is located in a renovated 100 year o...","""Bonterra""","""AMEX, Diners Club, Discover, MasterCard, Visa""","""$31 to $50""","""1829 Cleveland AvenueCharlotte, NC 28203""","""Charlotte""","""28203""","""NC""","""United States""",40.0,"Little old fashioned, mature crowd. Food was O...",4.075


In [10]:
# MERGE CITY SEARCH TERM INTO LOCALITY FOR MV IN LOCALITY (only 25 MV)

mapping = {"chicago":"Chicago","dallas":"Dallas","houston":"Houston","las-vegas":"Las Vegas","los-angeles":"Los Angeles",
          "new-york":"New York","san-antonio":"San Antonio","san-francisco":"San Francisco", "scottsdale":"Scottsdale"}

#sample[sample['Locality'].isnull()]
sample.loc[sample['Locality'].isnull(), 'Locality'] = sample.loc[sample['Locality'].isnull(), 'City'].map(mapping)

sample.drop(labels=['City'], axis=1, inplace=True)

In [11]:
sample.head()

,Cuisine,Description,Name,PaymentMethods,Pricerange,Street,Locality,Postalcode,Region,Country,ReviewCount,ReviewBodies,AverageRating
1131,"""American""","""JP Charlotte is at the heart of The Westin Ch...","""JP Charlotte""","""AMEX, Carte Blanche, Diners Club, Discover, J...","""$30 and under""","""601 S College StreetCharlotte, NC 28202""","""Charlotte""","""28202""","""NC""","""United States""",40.0,We traveled to Uptown Charlotte for Easter Bru...,3.850
1132,"""Steakhouse""","""Morton's The Steakhouse, the nation's premier...","""Morton's The Steakhouse - Charlotte""","""AMEX, Diners Club, MasterCard, Visa""","""$50 and over""","""227 W. Trade StreetCharlotte, NC 28202""","""Charlotte""","""28202""","""NC""","""United States""",40.0,"Our go-to for fine dining,steak and lobster. W...",4.450
1142,"""American""","""Sullivan\u2019s is a vibrant neighborhood Ame...","""Sullivan's Steakhouse - Charlotte""","""AMEX, Diners Club, Discover, MasterCard, Visa""","""$31 to $50""","""1928 South BoulevardSuite 200Charlotte, NC 28...","""Charlotte""","""28203""","""NC""","""United States""",40.0,Waited a while to get main course and my steak...,3.925
1145,"""American""","""Red Rocks Caf\u00E9 is a locally owned and op...","""Red Rocks Cafe - Charlotte""","""AMEX, MasterCard, Visa""","""$30 and under""","""4223-8 Providence RoadCharlotte, NC 28211""","""Charlotte""","""28211""","""NC""","""United States""",40.0,We dined outdoors and the device and good were...,4.375
1146,"""Contemporary American""","""Bonterra is located in a renovated 100 year o...","""Bonterra""","""AMEX, Diners Club, Discover, MasterCard, Visa""","""$31 to $50""","""1829 Cleveland AvenueCharlotte, NC 28203""","""Charlotte""","""28203""","""NC""","""United States""",40.0,"Little old fashioned, mature crowd. Food was O...",4.075


In [12]:
sample.to_json("schemaOrg.json", orient="records")

In [13]:
ola = sample.to_json(orient="records")